In [3]:
!pip install gym > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install h5py pyyaml > /dev/null 2>&1
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
!unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

     |████████████████████████████████| 946 kB 26.8 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Importing Dependancies


In [11]:
import keras
from keras.callbacks import TensorBoard
import os.path
import time
import shutil
from tensorflow.keras.optimizers import RMSprop
from keras import backend as K
from keras.models import load_model
from collections import deque 
from datetime import datetime
from keras.models import clone_model
from skimage.color import rgb2gray
from skimage.transform import resize
import gym
import tensorflow as tf
import numpy as np
import random
import math
import glob
import io
import base64

# Hyperparameters

In [12]:
observe_step_num = 10000
batch_size = 32
gamma = 0.99
replay_memory = 350000
num_episode = 100000
learning_rate = 0.00025
init_epsilon = 1.0
final_epsilon = 0.1
epsilon_step_num = 1000000
refresh_target_model_num = 10000
no_op_steps = 30
train_dir = "training_dir"
tensorboard_dir = "logs"
resume = False
!mkdir logs
!mkdir training_dir

mkdir: cannot create directory ‘logs’: File exists
mkdir: cannot create directory ‘training_dir’: File exists


**Preprocessing Function**

In [13]:
def pre_processing(observe):
    processed_observe = np.uint8(
        resize(rgb2gray(observe), (84 , 84), mode='constant') * 255)
    return processed_observe

**Huber loss function**

In [14]:
def huber_loss(y, q_value):
    error = K.abs(y - q_value)
    quadratic_part = K.clip(error, 0.0, 1.0)
    linear_part = error - quadratic_part
    loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)
    return loss

# DQN model

In [15]:
ATARI_SHAPE = (84, 84, 4)
ACTION_SIZE = 4

def atari_model():
    
    # Input layers.
    frames_input = keras.layers.Input(ATARI_SHAPE, name='frames')
    actions_input = keras.layers.Input((ACTION_SIZE,), name='mask')

    # Assuming that the input frames are still encoded from 0 to 255. Transforming to [0, 1].
    normalized = keras.layers.Lambda(lambda x: x / 255.0)(frames_input)
    
    # "The first hidden layer convolves 32 8×8 filters with stride 4 with the input image and applies a rectifier nonlinearity."
    conv_1 = keras.layers.Conv2D(
        32, (8, 8), strides=4, activation='relu', name="Conv1"
    )(normalized)
    
    # "The second hidden layer convolves 64 4×4 filters with stride 2, followed by a rectifier nonlinearity."
    conv_2 = keras.layers.Conv2D(
        64, (4, 4), strides=2, activation='relu', name="Conv2"
    )(conv_1)
    
    # "The third hidden layer convolves 64 3x3 filters with stride 1, again followed by a rectifier nonlinearity."
    conv_3 = keras.layers.Conv2D(
        64, (3, 3), strides=1, activation='relu', name="Conv3"
    )(conv_2)
    
    # Flattening the convolutional layer.
    conv_flattened = keras.layers.core.Flatten(name="flatten")(conv_3)
    
    # "Fully connected layer made up of 512 rectifier units."
    hidden = keras.layers.Dense(512, activation='relu', name="Dense512")(conv_flattened)
    
    # The output layer is a fully-connected linear layer with a single output for each valid action."
    output = keras.layers.Dense(ACTION_SIZE, name="Output")(hidden)
    
    # Finally, we multiply the output by the mask.
    filtered_output = keras.layers.Multiply(name='QValue')([output, actions_input])

    model = keras.models.Model(input=[frames_input, actions_input], output=filtered_output)
    optimizer = keras.optimizers.RMSprop(lr=0.00025, rho=0.95, epsilon=0.01)
    model.compile(optimizer, loss=huber_loss)
    model.summary()
    return model
atari_model()

TypeError: ignored